In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [9]:
def collect_vacancies_hh(profession):
    headers = {'User-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                             "AppleWebKit/537.36 (KHTML, like Gecko) "
                             "Chrome/98.0.4758.82 Safari/537.36"}

    profession = profession.replace(' ', '+')
    params = "/search/vacancy?clusters=true&ored_clusters=true" \
             "&enable_snippets=true&salary=&text=" + profession

    vacancies_list = {'Name':[],
                      'Source':[],
                      'Minimum Salary':[],
                      'Maximum Salary':[],
                      'Currency':[],
                      'Link':[]}
    while True:
        url = "https://novyj-urengoy.hh.ru" + params
        response = requests.get(url, headers=headers)
        
        
        if response.ok:
            dom = BeautifulSoup(response.text, 'html.parser')
            vacancies = dom.find_all("div", {"class": "vacancy-serp-item"})
            for vacancy in vacancies:
                vacancy_title = vacancy.find('a', {
                    'data-qa': 'vacancy-serp__vacancy-title'})
                vacancy_compensation = vacancy.find('span', {
                    'data-qa': 'vacancy-serp__vacancy-compensation'})
                
                vacancies_list['Name'].append(vacancy_title.getText())
                vacancies_list['Source'].append('hh.ru')
                
                min_wg, max_wg, cur = None, None, None
                if vacancy_compensation != None:
                    salary_info = vacancy_compensation.getText().split()
                    min_wg, max_wg, cur = read_compensation(salary_info)
                vacancies_list['Minimum Salary'].append(min_wg)
                vacancies_list['Maximum Salary'].append(max_wg)
                vacancies_list['Currency'].append(cur)
                    
                vacancies_list['Link'].append(vacancy_title['href'])

            #проверка на наличие кнопки "дальше"
            button = dom.find('a', {'data-qa':"pager-next"})
            if button == None:
                return vacancies_list
            params = button.attrs['href']

In [3]:
def read_compensation(contents):
    min_wage = None
    max_wage = None
    currency = contents[-1].replace('.','')

    if contents[0] == 'до':
        max_wage = contents[1] + contents[2]
    elif contents[0] == 'от':
        min_wage = contents[1] + contents[2]
    elif contents[2] == '–':
        min_wage = contents[0] + contents[1]
        max_wage = contents[3] + contents[4]

    return min_wage, max_wage, currency

In [10]:
profession = 'программист Python'
vac_list = collect_vacancies_hh(profession)

In [11]:
vacancies_df = pd.DataFrame(vac_list) 

In [12]:
print(vacancies_df)

                                                  Name Source Minimum Salary  \
0                                     Python Developer  hh.ru           2500   
1                 Software Engineer / Python developer  hh.ru           3000   
2                                    Разработчик C/C++  hh.ru           None   
3                             Backend Python Developer  hh.ru           None   
4                                 PHP/Python Developer  hh.ru          20000   
..                                                 ...    ...            ...   
795  Разработчик системы мониторинга оборудования и...  hh.ru           None   
796                 Middle Python Developer (удаленно)  hh.ru         260000   
797                 Разработчик DWH (Airflow + Python)  hh.ru           None   
798                            Senior Python Developer  hh.ru           None   
799                                 Программист Python  hh.ru           None   

    Maximum Salary Currency            